In [8]:
import json
import random
import requests
import time
from datetime import datetime, timedelta

from src.config import ORION_URL, NGSI_LD_ENDPOINT, headers
import src.utils  as ut

In [9]:
ut.test_connection()

Status Code: 200
Orion Version: 1.15.0-next
Orion-LD Version: 1.4.0


In [16]:
# Generate realistic charging duration 
def generate_charging_duration():
    """
    Generate realistic charging duration in minutes based on historical data
    基于历史数据生成真实的充电时长（分钟）
    """
    duration = random.normalvariate(225.97, 146.09)  
    return max(1, min(672, int(duration)))  

# Calculate energy consumption 
def calculate_energy(duration_minutes):
    """
    Calculate energy consumption based on duration using your formula
    根据时长使用你的公式计算能耗
    """
    energy = 0.0557 * duration_minutes + 3.3513  
    energy = energy * random.uniform(0.9, 1.1) 
    return round(max(0, min(energy, 56.5)), 3)  

# Generate random time session 
def generate_random_session(session_number):
    """
    Generate a charging session with completely random start time
    生成完全随机开始时间的充电会话
    """
    
    # Basic session information 
    timestamp = int(time.time() * 1000)
    session_id = f"{session_number:03d}_{timestamp}"
    
    # Charging point selection 
    charging_point_prefixes = ["fakeStation-1-P-", "fakeStation-2-P-", "fakeStation-3-P-"]
    charging_point_prefix = random.choice(charging_point_prefixes)
    charging_point_id = f"{charging_point_prefix}{random.randint(1, 10):02d}"
    
    # Random time across the whole year 2024 
    base_time = datetime(2024, 1, 1, 0, 0, 0)
    start_offset = timedelta(
        days=random.randint(0, 364),    
        hours=random.randint(0, 23),    
        minutes=random.randint(0, 59),  
        seconds=random.randint(0, 59)   
    )
    start_time = base_time + start_offset
    
    # Generate charging duration and energy consumption 
    charging_duration = generate_charging_duration()  
    energy_consumed = calculate_energy(charging_duration)  
    
    # Calculate end time / 计算结束时间
    end_time = start_time + timedelta(minutes=charging_duration)
    
    # Build the NGSI-LD entity structure 
    entity = {
        "id": f"urn:ngsi-ld:ChargingSessionFake:session-{session_id}",
        "type": "ChargingSessionFake",
        
        "refChargingPoint": {
            "type": "Relationship",
            "object": f"urn:ngsi-ld:ChargingPoint:{charging_point_id}"
        },
        
        "sessionId": {
            "type": "Property",
            "value": session_id
        },
        
        "sessionStatus": {
            "type": "Property",
            "value": "ended"  # All sessions are completed 
        },
        
        "sessionStartTime": {
            "type": "Property",
            "value": {
                "@type": "DateTime",
                "@value": start_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")
            },
            "observedAt": start_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")
        },
        
        "sessionEndTime": {
            "type": "Property",
            "value": {
                "@type": "DateTime",
                "@value": end_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")
            },
            "observedAt": end_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")
        },
        
        "chargingDuration": {
            "type": "Property",
            "value": charging_duration,  # Duration in minutes 
            "unitCode": "MIN"
        },
        
        "energyConsumed": {
            "type": "Property",
            "value": energy_consumed,  # Energy consumption in kWh 
            "unitCode": "KWH"
        }
    }
    
    return entity

# Generate time slot distributed session 
def generate_time_slot_session(session_number):
    """
    Generate a charging session with time slot distribution
    生成按时间段分布的充电会话
    
    Time slots distribution 
    - morning (7:00-11:00): 20% 
    - daytime (11:00-18:00): 30% 
    - evening (18:00-22:00): 20% 
    - night (22:00-07:00): 30% 
    """
    
    # Basic session information 
    timestamp = int(time.time() * 1000)
    session_id = f"{session_number:03d}_{timestamp}"
    
    # Charging point selection 
    charging_point_prefixes = ["fakeStation-1-P-", "fakeStation-2-P-", "fakeStation-3-P-"]
    charging_point_prefix = random.choice(charging_point_prefixes)
    charging_point_id = f"{charging_point_prefix}{random.randint(1, 10):02d}"
    
    # Select time slot based on distribution / 根据分布选择时间段
    time_slots = ['morning', 'daytime', 'evening', 'night']
    weights = [20, 30, 20, 30]  # Corresponding percentages / 对应的百分比
    slot = random.choices(time_slots, weights=weights)[0]
    
    # Random date in 2024 
    base_time = datetime(2024, 1, 1, 0, 0, 0)
    random_date = base_time + timedelta(days=random.randint(0, 364))
    
    # Set start time based on time slot / 根据时间段设置开始时间
    if slot == 'morning':      # Morning: 7:00-11:00 
        hour = random.randint(7, 10)
    elif slot == 'daytime':    # Daytime: 11:00-18:00
        hour = random.randint(11, 17)
    elif slot == 'evening':    # Evening: 18:00-22:00 
        hour = random.randint(18, 21)
    else:  # night             # Night: 22:00-07:00 
        hour = random.choice(list(range(22, 24)) + list(range(0, 7)))
    
    start_time = random_date.replace(
        hour=hour,
        minute=random.randint(0, 59),
        second=random.randint(0, 59)
    )
    
    # Generate charging duration and energy consumption 
    charging_duration = generate_charging_duration()  
    energy_consumed = calculate_energy(charging_duration)  
    
    # Calculate end time 
    end_time = start_time + timedelta(minutes=charging_duration)
    
    # Build the NGSI-LD entity structure / 构建NGSI-LD实体结构
    entity = {
        "id": f"urn:ngsi-ld:ChargingSessionFake:session-{session_id}",
        "type": "ChargingSessionFake",
        
        "refChargingPoint": {
            "type": "Relationship",
            "object": f"urn:ngsi-ld:ChargingPoint:{charging_point_id}"
        },
        
        "sessionId": {
            "type": "Property",
            "value": session_id
        },
        
        "sessionStatus": {
            "type": "Property",
            "value": "ended"  # All sessions are completed 
        },
        
        "sessionStartTime": {
            "type": "Property",
            "value": {
                "@type": "DateTime",
                "@value": start_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")
            },
            "observedAt": start_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")
        },
        
        "sessionEndTime": {
            "type": "Property",
            "value": {
                "@type": "DateTime",
                "@value": end_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")
            },
            "observedAt": end_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")
        },
        
        "chargingDuration": {
            "type": "Property",
            "value": charging_duration,  # Duration in minutes 
            "unitCode": "MIN"
        },
        
        "energyConsumed": {
            "type": "Property",
            "value": energy_consumed,  # Energy consumption in kWh 
            "unitCode": "KWH"
        }
    }
    
    return entity

# Batch generate sessions / 批量生成会话
def generate_sessions(count, session_type="random"):
    """
    Generate multiple charging sessions in batch
    批量生成多个充电会话
    
    Parameters / 参数:
    - count: number of sessions to generate / 要生成的会话数量
    - session_type: "random" or "time_slot" / "random" 或 "time_slot"
    """
    sessions = []
    
    for i in range(1, count + 1):
        if session_type == "time_slot":
            session = generate_time_slot_session(i)
        else:
            session = generate_random_session(i)
        
        sessions.append(session)
        time.sleep(0.001)  # Avoid timestamp duplication / 避免时间戳重复
    
    return sessions

# Usage examples / 使用示例
if __name__ == "__main__":
    # Generate 10 random time sessions / 生成10个随机时间的会话
    print("=== Random Time Sessions / 随机时间会话 ===")
    random_sessions = generate_sessions(10, "random")
    for i, session in enumerate(random_sessions[:3]):  # Display first 3 / 显示前3个
        print(f"Session {i+1}:")
        print(f"  ID: {session['sessionId']['value']}")
        print(f"  Start Time / 开始时间: {session['sessionStartTime']['value']['@value']}")
        print(f"  Duration / 时长: {session['chargingDuration']['value']} minutes / 分钟")
        print(f"  Energy / 能耗: {session['energyConsumed']['value']} kWh")
        print(f"  Charging Point / 充电桩: {session['refChargingPoint']['object']}")
        print("-" * 50)
    
    print("\n=== Time Slot Distributed Sessions / 时间段分布会话 ===")
    time_sessions = generate_sessions(10, "time_slot")
    for i, session in enumerate(time_sessions[:3]):  # Display first 3 / 显示前3个
        print(f"Session {i+1}:")
        print(f"  ID: {session['sessionId']['value']}")
        print(f"  Start Time / 开始时间: {session['sessionStartTime']['value']['@value']}")
        print(f"  Duration / 时长: {session['chargingDuration']['value']} minutes / 分钟")
        print(f"  Energy / 能耗: {session['energyConsumed']['value']} kWh")
        print(f"  Charging Point / 充电桩: {session['refChargingPoint']['object']}")
        print("-" * 50)

=== Random Time Sessions / 随机时间会话 ===
Session 1:
  ID: 001_1753720787566
  Start Time / 开始时间: 2024-08-31T19:27:33.000Z
  Duration / 时长: 1 minutes / 分钟
  Energy / 能耗: 3.308 kWh
  Charging Point / 充电桩: urn:ngsi-ld:ChargingPoint:fakeStation-3-P-07
--------------------------------------------------
Session 2:
  ID: 002_1753720787567
  Start Time / 开始时间: 2024-12-29T03:44:23.000Z
  Duration / 时长: 131 minutes / 分钟
  Energy / 能耗: 9.916 kWh
  Charging Point / 充电桩: urn:ngsi-ld:ChargingPoint:fakeStation-3-P-06
--------------------------------------------------
Session 3:
  ID: 003_1753720787569
  Start Time / 开始时间: 2024-01-10T15:22:31.000Z
  Duration / 时长: 270 minutes / 分钟
  Energy / 能耗: 18.946 kWh
  Charging Point / 充电桩: urn:ngsi-ld:ChargingPoint:fakeStation-2-P-07
--------------------------------------------------

=== Time Slot Distributed Sessions / 时间段分布会话 ===
Session 1:
  ID: 001_1753720787578
  Start Time / 开始时间: 2024-11-04T11:41:38.000Z
  Duration / 时长: 276 minutes / 分钟
  Energy / 能耗: 17.

In [11]:
def generate_multiple_sessions(count=10):
    """
    Generate multiple charging sessions
    生成多个充电会话
    """
    sessions = []
    
    for i in range(1, count + 1):
        # Small delay to ensure unique timestamps
        time.sleep(0.001)
        session = generate_random_fake_session(i)
        sessions.append(session)
    
    return sessions

def show_statistics(sessions):
    """
    Display basic statistics of generated sessions
    显示生成会话的基本统计信息
    """
    durations = [s['chargingDuration']['value'] for s in sessions]
    energies = [s['energyConsumed']['value'] for s in sessions]
    
    print(f"Generated {len(sessions)} sessions")
    print(f"Duration: min={min(durations)}, max={max(durations)}, avg={sum(durations)/len(durations):.1f} min")
    print(f"Energy: min={min(energies):.2f}, max={max(energies):.2f}, avg={sum(energies)/len(energies):.2f} kWh")
    
    # Compare to historical
    print(f"\nHistorical vs Generated:")
    print(f"Duration avg: 225.97 vs {sum(durations)/len(durations):.1f}")
    print(f"Energy avg: 15.95 vs {sum(energies)/len(energies):.2f}")

def verify_formula(sessions, sample_size=5):
    """
    Verify the energy calculation formula
    验证能耗计算公式
    """
    print(f"\n=== Formula Verification (first {sample_size} sessions) ===")
    for i in range(min(sample_size, len(sessions))):
        session = sessions[i]
        duration = session['chargingDuration']['value']
        actual_energy = session['energyConsumed']['value']
        expected_energy = 0.0557 * duration + 3.3513
        
        print(f"Session {i+1}: {duration} min → {actual_energy:.3f} kWh (expected: {expected_energy:.3f})")

In [12]:
# Generate single session
session = generate_random_fake_session(1)
print(f"\nSingle session:")
print(f"ID: {session['sessionId']['value']}")
print(f"Duration: {session['chargingDuration']['value']} min")
print(f"Energy: {session['energyConsumed']['value']} kWh")
print(f"Charging Point: {session['refChargingPoint']['object']}")



Single session:
ID: 001_1753716056132
Duration: 305 min
Energy: 21.34 kWh
Charging Point: urn:ngsi-ld:ChargingPoint:fakeStation-2-P-01


In [13]:
# Generate multiple sessions
sessions = generate_multiple_sessions(20)
show_statistics(sessions)

# Verify formula
verify_formula(sessions)


Generated 20 sessions
Duration: min=1, max=441, avg=225.3 min
Energy: min=3.17, max=29.42, avg=16.20 kWh

Historical vs Generated:
Duration avg: 225.97 vs 225.3
Energy avg: 15.95 vs 16.20

=== Formula Verification (first 5 sessions) ===
Session 1: 367 min → 26.171 kWh (expected: 23.793)
Session 2: 441 min → 27.055 kWh (expected: 27.915)
Session 3: 232 min → 15.941 kWh (expected: 16.274)
Session 4: 235 min → 16.416 kWh (expected: 16.441)
Session 5: 144 min → 11.890 kWh (expected: 11.372)


In [14]:
# Show full structure of first session
print(json.dumps(sessions[0], indent=2))

{
  "id": "urn:ngsi-ld:ChargingSessionFake:session-001_1753716056156",
  "type": "ChargingSessionFake",
  "refChargingPoint": {
    "type": "Relationship",
    "object": "urn:ngsi-ld:ChargingPoint:fakeStation-2-P-03"
  },
  "sessionId": {
    "type": "Property",
    "value": "001_1753716056156"
  },
  "sessionStatus": {
    "type": "Property",
    "value": "ended"
  },
  "sessionStartTime": {
    "type": "Property",
    "value": {
      "@type": "DateTime",
      "@value": "2024-08-28T18:03:05.000Z"
    },
    "observedAt": "2024-08-28T18:03:05.000Z"
  },
  "sessionEndTime": {
    "type": "Property",
    "value": {
      "@type": "DateTime",
      "@value": "2024-08-29T00:10:05.000Z"
    },
    "observedAt": "2024-08-29T00:10:05.000Z"
  },
  "chargingDuration": {
    "type": "Property",
    "value": 367,
    "unitCode": "MIN"
  },
  "energyConsumed": {
    "type": "Property",
    "value": 26.171,
    "unitCode": "KWH"
  }
}
